In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
# encoders and decoders = tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
# training and validation splits
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data= train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(f'\nsize x matrix {x.shape}\n')
print('inputs: ')
print(x)
print('targets: ')
print(y)


size x matrix torch.Size([4, 8])

inputs: 
tensor([[58, 67, 72, 62, 68, 67, 72, 11],
        [61, 58,  0, 36, 54, 67, 57,  1],
        [ 1, 73, 68,  0, 60, 71, 54, 67],
        [75, 58, 71, 78, 68, 67, 58,  1]])
targets: 
tensor([[67, 72, 62, 68, 67, 72, 11,  1],
        [58,  0, 36, 54, 67, 57,  1, 68],
        [73, 68,  0, 60, 71, 54, 67, 62],
        [58, 71, 78, 68, 67, 58,  1, 56]])


In [5]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('when input is', context, 'target is ', target)

# how to measure the loss:
# -ln(1/80) = to predict the next carachter. // have to minimize the loss, the objective
# utilize adamW as the optimize algorithm

In [6]:
@torch.no_grad() # certificates that pytorch doesn't use any gradient here
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):

    def __init__ (self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits =  self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the laste time step
            logits = logits[:, -1, :] # becomes(B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



2,WxKHT;MydeBw5M!JchFZHi(z1E3XB;7mYOv'XXv.I(?
S sWQI'Fv'(lpV7(lluuGWYgI:OF-owv&vf3hOl 9XZwhk.Z'y"yOye5G
K?QIkRYT MMC(o8EguHuaQp8W14"FN3cDUZ.z_1ca!f:B26f1_C";wV-)*"o6C'4GWtO]Q1:!d3h7-qHTE&bp,hxst5Z:KjS,3?2nOhxb?.n[PbHiRjUd9TPQxaZv;ERWYDZIWtLtMuGy,mO-HTP K*A*7Mu101'odqJU3LFyxP
'MiW;s7-3"-WxCyDmItMS"wrJanM[Ks_"GIbOmRDp01,DT:p6o K6ft:J"kYYRDdxIWXi.A7MKzzPnffKYDTyts&OLCX9gQuchPruorI[rBj*aHT"GqzctMGtskZauGDlZtiRfaXmaPkQ1PC5_(FPIgS,m(1'f- "1S4)X3'vvps27.ZzerLT7-?P5SJT*"nF8gRbKpS U KHicFw"bLiw46:&:pp8pG


In [8]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range (max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')


    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logit, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train loss: 4.9956, val loss: 4.9618
step: 250, train loss: 4.9324, val loss: 4.8953
step: 500, train loss: 4.8771, val loss: 4.8400
step: 750, train loss: 4.8099, val loss: 4.7965
4.872916221618652


In [9]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


i3cGuYmLC5*DTu18TI[Cml?.x)_R*m_CWkK26Ce8B?K2(JPb?AVC5[D,g.7Yq[9G0A.iFqpDM7-*d[w7Sv&uRrizR5irgvt] V'fgR-"S7-',[!4 .2zr?,
-6)5-yoUD1i?*d[5foch6]-J1T!tuh'88lX33LqcgRv?Rbf0E&,6iP"Cxmx]FDrgI-,wt0Y]F[raEg9.A[lTrUbfOMkh(Q!&S7P".Jrj[Z-U3[o8vfC:uxgBRUQ.dEBaT)O"gjM!_DknZwil7-xPLo(lG)BaE[9fj'h.L4"
g9boVtXf-QAp,33LYsOWd:26C'V;O3CuGW;wcGLY_JLqs_p
dM1c-8"ZU':Z[9TJ26nUZIJ2:s]BU(ZI-vKROE&zfrD a,&Evpe]mQ7xZ7G0;h.!t[9t4"2"5W1TiG*b!XZa!W39-*b).Z?bM14GWK"Y]j7pA;lhp&eh'M!t0YU3Ux(zqp'H2?.imK5d[PyoUVwmXo8HrDr'(zb1d3gs
